In [1]:
import torch
import numpy as np
import datetime
import random
import time
#from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset,RandomSampler,SequentialSampler
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from accelerate import Accelerator

/home/kbudakovskiy/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
accelerator = Accelerator()
device = accelerator.device
# device = "cpu"

In [3]:
model_name="ai-forever/ruBert-base"

tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,                       
    output_attentions=False,
    output_hidden_states=False,
)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [4]:
dataset_train_fn = 'train.jsonl'
dataset_validation_fn = "val.jsonl"
dataset_test_fn = 'test.jsonl'

In [5]:
import pandas as pd
train_df = pd.read_json(dataset_train_fn,lines=True)
validation_df = pd.read_json(dataset_validation_fn,lines=True)
test_df = pd.read_json(dataset_test_fn,lines=True)

In [6]:
train_df.set_index('idx', inplace=True)
validation_df.set_index('idx', inplace=True)
test_df.set_index('idx', inplace=True)

In [7]:
train_df.head()

,word,sentence1,sentence2,start1,end1,start2,end2,label,gold_sense1,gold_sense2
idx,,,,,,,,,,
0,двор,В нашей деревне осталось от силы двадцать дворов,"Солнце стояло уже высоко, когда справа от доро...",42,49,69,76,True,1,1
1,доклад,Табличка на дверях: «Без доклада не входить»,Вчера отбыл в столицу первый секретарь обкома ...,25,33,97,104,False,4,2
2,засада,"У нас вообще […] засада с героями, способными ...",Там в воскресенье все магазины закрыты – вот в...,17,24,50,57,True,2,2
3,доля,"Он не успел сказать и десятой доли того, что с...",Болезнь ее была странного свойства – […] что-т...,30,35,70,77,False,2,3
4,закат,"Теперь, если она не пойдет звонить мужу, успее...","Тридцать с лишним лет службы в органах, три ра...",51,58,85,92,False,1,2


In [8]:
max_len = 0
def max_compound_string_lenght_in_dataset(row, max_len) -> int:
    # токенизируем текст, не забывая добавлять специальный токен начала и конца([CLS] и [SEP])
    train_input_ids = tokenizer.encode(
        row["sentence1"]+ ". " + row["sentence2"]+ ". " + row["word"],add_special_tokens=True)
    # Сравниваем длину последовательности и если она длиннее - обновляем max_len
    return max(max_len, len(train_input_ids))

for df in [test_df, train_df, validation_df]:
    for _, row in df.iterrows():
        max_len = max_compound_string_lenght_in_dataset(row, max_len)

print('Максимальная длина строки: ', max_len)



Максимальная длина строки:  164


In [9]:
def encode(sentences: pd.Series) -> list | list:
    input_ids = []
    attention_masks = []
    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sent,                           # Предложение для кодирования.
                            add_special_tokens = True,      # Добавить '[CLS]' и '[SEP]'.
                            max_length = 180,               # Дополнить и обрезать все предложения.
                            pad_to_max_length = True,       # Дополнить до максимальной длины и обрезать.
                            return_attention_mask = True,   # Создать маски внимания.
                            return_tensors = 'pt',          # Вернуть тензоры PyTorch.
                    )

        # Добавляем закодированное предложение в список.
        input_ids.append(encoded_dict['input_ids'])

        # А также его маску внимания (просто отличает заполнение от незаполненного).
        attention_masks.append(encoded_dict['attention_mask'])

    # Возвращаем списки тензоров для input_ids и attention_masks.
    return input_ids, attention_masks

In [10]:
train_sentences=train_df["sentence1"]+ ". " + train_df["sentence2"]+ ". " + train_df["word"]
train_sentences=train_sentences.values
validation_sentences=validation_df["sentence1"]+ ". " + validation_df["sentence2"]+ ". " +validation_df["word"]
validation_sentences=validation_sentences.values
test_sentences=test_df["sentence1"]+ ". " + test_df["sentence2"]+ ". " +test_df["word"]
test_sentences=test_sentences.values

train_labels=[1 if x else 0 for x in train_df.label.values]
validation_labels=[1 if x else 0 for x in validation_df.label.values]



In [11]:
# Токенизируем все предложения и сопоставляем токены их идентификаторам слов.
train_input_ids, train_attention_masks = encode(train_sentences)
validation_input_ids, validation_attention_masks = encode(validation_sentences)
test_input_ids, test_attention_masks = encode(test_sentences)

# Преобразуем списки в тензоры.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

validation_input_ids = torch.cat(validation_input_ids, dim=0)
validation_attention_masks = torch.cat(validation_attention_masks, dim=0)
validation_labels = torch.tensor(validation_labels)

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

# Выводим первое предложение как список идентификаторов слов.
print('Исходное предложение: ', train_sentences[0])
print('Идентификаторы слов:', train_input_ids[0])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/kbudakovskiy/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Исходное предложение:  В нашей деревне осталось от силы двадцать дворов. Солнце стояло уже высоко, когда справа от дороги я увидел деревеньку дворов в пятнадцать. двор
Идентификаторы слов: tensor([  101,   113,  5937,   378,  8278,  5493,   700,  3372,  6290, 17933,
          126,  8072, 24826,   965,  4396,   121,  1040, 12922,   700,  4361,
          119,  4400, 22818,   733, 17933,   113, 14222,   126,  4883,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

In [12]:
# Создаем TensorDataset из тензоров input_ids, attention_masks и labels.
train_dataset=TensorDataset(train_input_ids, train_attention_masks, train_labels)
validation_dataset=TensorDataset(validation_input_ids, validation_attention_masks, validation_labels)


In [13]:
# DataLoader должен знать размер нашего батча для обучения, поэтому мы указываем его здесь.
# При fine-tuning BERT на конкретной задаче авторы рекомендуют размер батча 16 или 32.
batch_size = 32

# Создаем DataLoaders для наших обучающих и валидационных наборов данных.
# Образцы для обучения будем брать в случайном порядке.
train_dataloader = DataLoader(
            train_dataset,  # Обучающие образцы.
            sampler = RandomSampler(train_dataset), # Выбираем батчи случайным образом.
            batch_size = batch_size # Обучение с этим размером батча.
        )

# Для валидации порядок не имеет значения, поэтому мы просто читаем их последовательно.
validation_dataloader = DataLoader(
            validation_dataset, # Валидационные образцы.
            sampler = SequentialSampler(validation_dataset), # Извлекаем батчи последовательно.
            batch_size = batch_size # Оценка с этим размером батча.
        )

In [14]:
# Примечание: AdamW - это класс из библиотеки huggingface (в отличие от pytorch)
# Вероятно, 'W' означает 'Weight Decay fix' (исправление весового распада, что есть не очень понятно по звучанию).

# Инициализируем оптимизатор AdamW.
optimizer = torch.optim.AdamW(model.parameters(),
                  lr=2e-5,   # Скорость обучения - по умолчанию 5e-5, в нашем случае 2e-5.
                  eps=1e-8   # Epsilon для Adam - по умолчанию 1e-8.
                )


In [15]:
# Количество эпох обучения. Авторы BERT рекомендуют от 2 до 4 эпох.
# Мы выбрали 4, но позже увидим, что это может привести к переобучению.

epochs = 4

# Общее количество шагов обучения - [количество батчей] x [количество эпох].
# (Обратите внимание, что это не то же самое, что количество обучающих образцов).
total_steps = len(train_dataloader) * epochs

# Создаем планировщик скорости обучения.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, # Значение по умолчанию в run_glue.py.
                                            num_training_steps=total_steps)

In [16]:
# Функция для вычисления точности наших предсказаний по сравнению с метками.
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [18]:
import os
output_dir = './model_save/'

# Создаем директорию в случае отсутствия
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [19]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
for epoch_i in range(0, epochs):
    print('\nОбучение: эпохи {:} / {:}:'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Батч {:>5,}  из  {:>5,}.    Затраченное время: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        res = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = res['loss']
        logits = res['logits']
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("\n  Средняя обучающая потеря: {0:.2f}".format(avg_train_loss))
    print(" Время затраченное обучение: {:}".format(training_time))
    print("\nВалидация")

    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            res = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = res['loss']
        logits = res['logits']
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Точность: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    print("  Потери валидации: {0:.2f}".format(avg_val_loss))
    print("  Валидация заняла: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Обучающая потеря': avg_train_loss,
            'Потери на валидации': avg_val_loss,
            'Точность на валидации': avg_val_accuracy,
            'Время обучения': training_time,
            'Время валидации': validation_time
        }
    )
print("\nОбучение завершено")
print("Обучение заняло {:} (ч:м:с)".format(format_time(time.time()-total_t0)))


Обучение: эпохи 1 / 4:
  Батч    50  из    621.    Затраченное время: 0:02:46.
  Батч   100  из    621.    Затраченное время: 0:05:00.
  Батч   150  из    621.    Затраченное время: 0:07:18.
  Батч   200  из    621.    Затраченное время: 0:09:32.
  Батч   250  из    621.    Затраченное время: 0:11:48.
  Батч   300  из    621.    Затраченное время: 0:14:11.
  Батч   350  из    621.    Затраченное время: 0:16:49.
  Батч   400  из    621.    Затраченное время: 0:19:27.
  Батч   450  из    621.    Затраченное время: 0:22:01.
  Батч   500  из    621.    Затраченное время: 0:24:35.
  Батч   550  из    621.    Затраченное время: 0:27:14.
  Батч   600  из    621.    Затраченное время: 0:29:50.

  Средняя обучающая потеря: 0.58
 Время затраченное обучение: 0:30:54

Валидация
  Точность: 0.76
  Потери валидации: 0.49
  Валидация заняла: 0:04:26

Обучение: эпохи 2 / 4:
  Батч    50  из    621.    Затраченное время: 0:02:35.
  Батч   100  из    621.    Затраченное время: 0:05:10.
  Батч   150  из

In [26]:
print("Сохранение модели в %s" % output_dir)
model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

Сохранение модели в ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

### Оценка на тесте

In [21]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)

In [22]:
test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            shuffle=False
        )

In [23]:
# Prediction on test set
print('Проверка модели {:,} на ткстовых данных'.format(len(test_input_ids)))
# Put model in evaluation mode
model.eval()
# Tracking variables
predictions , true_labels = [], []
# Predict
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()

  # Store predictions and true labels
  predictions.append(logits)
  
print('Заверешено')

Проверка модели 18,892 на ткстовых данных
Заверешено


In [24]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            shuffle=False
        )

print('Прогноз меток для {:,} тестовых предложений...'.format(len(test_input_ids)))
model.eval()
predictions = []

for idx, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0].cpu().numpy()
    
    predictions.extend([{"idx": int(idx), "label": logits[i].tolist()} for i in range(len(logits))])
print('Заверешено')


Прогноз меток для 18,892 тестовых предложений...
Заверешено


In [25]:
import json
output_file = 'RUSSE.jsonl'

print(f'Запись предсказанных значений в файл {output_file}')
with open(output_file, 'w') as json_file:
    for prediction in predictions:
        label=np.argmax(prediction["label"])
        if label==1:
            label="true"
        else:
            label="false"
        prediction["label"]=label
        json_file.write(json.dumps(prediction) + '\n')

Запись предсказанных значений в файл RUSSE.jsonl
